In [10]:
# set dependencies
import os
import csv
import requests
import time
import psycopg2
import pymongo
import json
import pprint
import urllib.request

import pandas as pd
import numpy as np

from flask_pymongo import PyMongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
from flask import Flask, render_template, redirect
from api_keys import nps_key

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows=100

# National Parks Visitor table

In [11]:
# Get data table
tables=pd.read_html('https://www.nps.gov/aboutus/visitation-numbers.htm', index_col=0)
table=pd.DataFrame(tables[1])
table

,Park,Recreational Visits
1,Great Smoky Mountains National Park,12.1 million
2,Yellowstone National Park,"3,8 million"
3,Zion National Park,3.6 million
4,Rocky Mountain National Park,3.3 million
5,Grand Teton National Park,3.3 million
6,Grand Canyon National Park,2.9 million
7,Cuyahoga Valley National Park,2.8 million
8,Acadia National Park,2.7 million
9,Olympic National Park,2.5 million
10,Joshua Tree National Park,2.4 million


# Kaggle National Parks

In [12]:
parks=pd.read_csv('dataFiles/parks.csv')
parks.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


# Kaggle National Parks species

In [13]:
species=pd.read_csv('dataFiles/species.csv',index_col=0,skipinitialspace=True,usecols=[1,2,3,4,5,6,7,8,9,10,11,12])
species.head()

,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status
Park Name,,,,,,,,,,,
Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN
Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN
Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern
Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered
Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN


# National Park Service API set-up

In [36]:
base_url='https://developer.nps.gov/api/v1'

# API routes for API looping
nps_api_list=['/activities','/activities/parks','/alerts','/amenities','/amenities/parksplaces',
              '/amenities/parksvisitorcenters','/articles','/campgrounds','/events','/lessonplans','/newsreleases','/parks',
              '/passportstamplocations','/people','/places','/thingstodo','/topics','/topics/parks','/tours','/visitorcenters',
              '/webcams']

# Name list for API looping
nps_api_names=['activities','activitiesparks','alerts','amenities','amenitiesparksplaces','amenitiesparksvisitorcenters',
               'articles','campgrounds','events','lessonplans','newsreleases','parks','passportstamplocations','people',
               'places','thingstodo','topics','topicsparks','tours','visitorcenters','webcams']

# Merge table and parks dataframes to reduce parks to just top 10, create code list and convert list to lowercase
table_merged=table.merge(parks,'left',left_on='Park',right_on='Park Name')
nps_park_codes= parks['Park Code'].tolist()
top_ten_park_codes=table_merged['Park Code'].tolist()
top_ten_park_codes=[x.lower() for x in top_ten_park_codes]

# Pretty print result
pprint.pformat(top_ten_park_codes,compact=True)
table_merged

,Park,Recreational Visits,Park Code,Park Name,State,Acres,Latitude,Longitude
0,Great Smoky Mountains National Park,12.1 million,GRSM,Great Smoky Mountains National Park,"TN, NC",521490,35.68,-83.53
1,Yellowstone National Park,"3,8 million",YELL,Yellowstone National Park,"WY, MT, ID",2219791,44.60,-110.50
2,Zion National Park,3.6 million,ZION,Zion National Park,UT,146598,37.30,-113.05
3,Rocky Mountain National Park,3.3 million,ROMO,Rocky Mountain National Park,CO,265828,40.40,-105.58
4,Grand Teton National Park,3.3 million,GRTE,Grand Teton National Park,WY,309995,43.73,-110.80
5,Grand Canyon National Park,2.9 million,GRCA,Grand Canyon National Park,AZ,1217403,36.06,-112.14
6,Cuyahoga Valley National Park,2.8 million,CUVA,Cuyahoga Valley National Park,OH,32950,41.24,-81.55
7,Acadia National Park,2.7 million,ACAD,Acadia National Park,ME,47390,44.35,-68.21
8,Olympic National Park,2.5 million,OLYM,Olympic National Park,WA,922651,47.97,-123.50
9,Joshua Tree National Park,2.4 million,JOTR,Joshua Tree National Park,CA,789745,33.79,-115.90


In [52]:
url=f'{base_url}/parks?parkCode=grsm,yell,zion,romo,grte,grca,cuva,acad,olym,jotr&api_key={nps_key}&limit=100'
response=requests.get(url)
top_ten_act=json.loads(response.content.decode('utf-8'))
top_ten_act=top_ten_act['data']
top_ten_act

[{'id': '6DA17C86-088E-4B4D-B862-7C1BD5CF236B',
  'url': 'https://www.nps.gov/acad/index.htm',
  'fullName': 'Acadia National Park',
  'parkCode': 'acad',
  'description': "Acadia National Park protects the natural beauty of the highest rocky headlands along the Atlantic coastline of the United States, an abundance of habitats, and a rich cultural heritage. At 3.5 million visits a year, it's one of the top 10 most-visited national parks in the United States. Visitors enjoy 27 miles of historic motor roads, 158 miles of hiking trails, and 45 miles of carriage roads.",
  'latitude': '44.409286',
  'longitude': '-68.247501',
  'latLong': 'lat:44.409286, long:-68.247501',
  'activities': [{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42',
    'name': 'Arts and Culture'},
   {'id': 'FAED7F97-3474-4C21-AB42-FB0768A2F826',
    'name': 'Cultural Demonstrations'},
   {'id': '13A57703-BB1A-41A2-94B8-53B692EB7238', 'name': 'Astronomy'},
   {'id': 'D37A0003-8317-4F04-8FB0-4CF0A272E195', 'name': 'Starg

In [16]:
# Capture data items
description=yell_data['description']
activities=[activity['name'] for activity in yell_data['activities']]
activities

['Arts and Culture',
 'Live Music',
 'Auto and ATV',
 'Scenic Driving',
 'Astronomy',
 'Stargazing',
 'Biking',
 'Road Biking',
 'Camping',
 'Backcountry Camping',
 'Car or Front Country Camping',
 'Horse Camping (see also Horse/Stock Use)',
 'Group Camping',
 'RV Camping',
 'Fishing',
 'Freshwater Fishing',
 'Fly Fishing',
 'Food',
 'Picnicking',
 'Guided Tours',
 'Self-Guided Tours - Auto',
 'Hands-On',
 'Citizen Science',
 'Hiking',
 'Backcountry Hiking',
 'Front-Country Hiking',
 'Horse Trekking',
 'Horse Camping (see also camping)',
 'Horseback Riding',
 'Junior Ranger Program',
 'Wildlife Watching',
 'Birdwatching',
 'Park Film',
 'Museum Exhibits',
 'Shopping',
 'Bookstore and Park Store']

In [18]:
df=pd.DataFrame.from_dict(yell_data,orient='index')
df

,0
id,D9819727-18DF-4A84-BDDE-D4F2696DE340
url,https://www.nps.gov/grsm/index.htm
fullName,Great Smoky Mountains National Park
parkCode,grsm
description,Ridge upon ridge of forest straddles the borde...
latitude,35.60116374
longitude,-83.50818326
latLong,"lat:35.60116374, long:-83.50818326"
activities,[{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...
topics,[]


In [51]:
all_table_merged=table_merged.merge(species,'left',on='Park Name')
all_table_merged

,Park,Recreational Visits,Park Code,Park Name,State,Acres,Latitude,Longitude,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status
0,Great Smoky Mountains National Park,12.1 million,GRSM,Great Smoky Mountains National Park,"TN, NC",521490,35.68,-83.53,Mammal,Rodentia,Cricetidae,Clethrionomys gapperi gapperi,Gapper's Red-Backed Vole,Approved,Not Present (False Report),Native,NaN,NaN,NaN
1,Great Smoky Mountains National Park,12.1 million,GRSM,Great Smoky Mountains National Park,"TN, NC",521490,35.68,-83.53,Mammal,Artiodactyla,Bovidae,Bos bison,"American Bison, Bison",Approved,Not Present (Historical Report),Native,NaN,NaN,NaN
2,Great Smoky Mountains National Park,12.1 million,GRSM,Great Smoky Mountains National Park,"TN, NC",521490,35.68,-83.53,Mammal,Artiodactyla,Bovidae,Bos taurus,"Aurochs, Aurochs, Domestic Cattle (Feral), Dom...",Approved,Not Present (Historical Report),Not Native,NaN,NaN,NaN
3,Great Smoky Mountains National Park,12.1 million,GRSM,Great Smoky Mountains National Park,"TN, NC",521490,35.68,-83.53,Mammal,Artiodactyla,Bovidae,Ovis aries,"Domestic Sheep, Mouflon, Red Sheep, Sheep (Feral)",Approved,Not Present (Historical Report),Not Native,NaN,NaN,NaN
4,Great Smoky Mountains National Park,12.1 million,GRSM,Great Smoky Mountains National Park,"TN, NC",521490,35.68,-83.53,Mammal,Artiodactyla,Cervidae,Cervus elaphus,Wapiti Or Elk,Approved,Present,Native,Rare,Breeder,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28076,Joshua Tree National Park,2.4 million,JOTR,Joshua Tree National Park,CA,789745,33.79,-115.90,Algae,NaN,NaN,Xanthonema debile,None,In Review,NaN,NaN,NaN,NaN,NaN
28077,Joshua Tree National Park,2.4 million,JOTR,Joshua Tree National Park,CA,789745,33.79,-115.90,Algae,Bacillariales,Bacillariaceae,Hantzschia amphioxys,None,In Review,NaN,NaN,NaN,NaN,NaN
28078,Joshua Tree National Park,2.4 million,JOTR,Joshua Tree National Park,CA,789745,33.79,-115.90,Algae,Naviculales,Diadesmidaceae,Luticola cohnii,None,In Review,NaN,NaN,NaN,NaN,NaN
28079,Joshua Tree National Park,2.4 million,JOTR,Joshua Tree National Park,CA,789745,33.79,-115.90,Algae,Naviculales,Diadesmidaceae,Luticola mutica,None,In Review,NaN,NaN,NaN,NaN,NaN
